In [0]:
rawdf1=spark.read.csv("/Volumes/lakehouse1/dbread/read_volume/custs",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession")
rawdf1.createOrReplaceTempView("v1")
spark.sql("select * from v1 where age >50").createOrReplaceTempView("_sqldf")

spark.sql("create or replace temp view v2 as select * from v1 where age <50")




In [0]:
%sql
select * from v2

In [0]:
rawdf1.printSchema()
print(rawdf1.columns)
rawdf1.dtypes
print(rawdf1.schema)

In [0]:
%sql
describe v1

In [0]:
rawdf1.createOrReplaceTempView("rawdf1view")
print("actual count of the data","select count(*) from rawdf1view")
print("de-duplicated record (all columns) count",rawdf1.distinct().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count",rawdf1.dropDuplicates().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated given cid column count",rawdf1.dropDuplicates(['id']).count())#de duplicate the entire columns of the given  dataframe
display(rawdf1.describe())
display(rawdf1.summary())

In [0]:
%sql
SELECT COUNT(*) AS dedup_all_columns_count 
FROM (SELECT distinct * FROM rawdf1view) t;

In [0]:
%sql
SELECT COUNT(*) AS dedup_all_columns_count 
FROM (SELECT id FROM rawdf1view where id is not null) t;

In [0]:
%sql

SELECT count(*) as cnt
FROM (
  SELECT *,row_number() over(partition by id order by id) as rno
  FROM rawdf1view
  --where rno=1
  ) inlineview
  where rno=1;


  select * from rawf1view qualify row_number() over(partition by id order by id)=1
    
select * from rawdf1view 

In [0]:
from pyspark.sql.session import SparkSession#15lakhs
spark=SparkSession.builder.appName("WD36 - ETL Pipeline - Bread & Butter").getOrCreate()#

In [0]:
#Extraction (Ingestion) methodologies
#1. Single file
struct1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(struct1).csv(path="/Volumes/lakehouse1/dbread/read_volume/custsmodified")
#2. Multiple files (with same or different names)
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/lakehouse1/dbread/read_volume/custsmodified","/Volumes/lakehouse1/dbread/read_volume/custsmodified"])
#3. Multiple files in multiple paths or sub paths
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/","/Volumes/workspace/wd36schema/ingestion_volume/staging/"],recursiveFileLookup=True,pathGlobFilter="custsm*")

In [0]:
#Validation by doing cleansing
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
#print(rawdf1.schema)
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/lakehouse1/dbread/read_volume/custsmodified",mode='permissive')
print("after keeping nulls on the wrong data format",cleandf1.count())#all rows count
display(cleandf1)#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato)
#or
#method2 - drop malformed rows
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/lakehouse1/dbread/read_volume/custsmodified",mode='dropMalformed')
print("after cleaning wrong data (type mismatch, column number mismatch)",len(cleandf1.collect()))
display(cleandf1)#We are removing the entire row, where ever data format mismatch is there (throwing away the entire potato)
print(cleandf1.count())#count will return the original count of the raw data
print(len(cleandf1.collect()))#collect+len will return the dropmalformed count of the raw data

In [0]:
struttype1=StructType([StructField('id', StringType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', StringType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
rawdf1=spark.read.schema(struttype1).csv(path="/Volumes/lakehouse1/dbread/read_volume/custsmodified",mode='permissive')
print("allow all data showing the real values",rawdf1.count())#all rows count
display(rawdf1)#

In [0]:
#Creating rejection dataset to send to our source system for future fix
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True),StructField("corruptedrows",StringType())])
#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/lakehouse1/dbread/read_volume/custsmodified",mode='permissive',columnNameOfCorruptRecord="corruptedrows")
#Create a reject dataset
rejectdf1=cleandf1.where("corruptedrows is not null")
#display(rejectdf1)
rejectdf1.write.csv("/Volumes/lakehouse1/dbread/read_volume/Rejected/",mode="overwrite",header=True)
retaineddf1=cleandf1.where("corruptedrows is null")
print("Overall rows in the source data is ",len(cleandf1.collect()))
print("Rejected rows in the source data is ",len(rejectdf1.collect()))
print("Clean rows in the source data is ",len(retaineddf1.collect()))